In [17]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Creamos las funciones para el Modelo de Recomendacion

## Abrimos la tabla procesada anteriormente

- random_merged_recommend_model.parquet

In [18]:
# Direccion del archivo comprimido y reconvertido
random_merged_recommend_content = 'FastAPI/merged_recommend_model.parquet'

# Cargar directamente el archivo comprimido en un DataFrame
random_merged_recommend_model = pd.read_parquet(random_merged_recommend_content)

## Funciones

In [19]:
# Filtrar los datos relevantes (recommend=True y sentimientos válidos)
relevant_data = random_merged_recommend_model[(random_merged_recommend_model['recommend'] == True) & 
                          random_merged_recommend_model['sentiment_analysis'].isin([0, 1, 2])]

# Construir la matriz de ítem-ítem usando pivot_table
item_item_matrix = relevant_data.pivot_table(index='app_name', columns='item_id', values='sentiment_analysis', fill_value=0)

# Calcular la similitud de coseno entre ítems
item_similarity = cosine_similarity(item_item_matrix.T)

# Función para obtener ítems similares a un ítem dado por su nombre ('app_name')
def get_similar_items(item_name, num_similar_items=5):
    item_id = item_item_matrix.index.get_loc(item_name)
    similar_indices = item_similarity[item_id].argsort()[::-1][1:num_similar_items+1]
    similar_items = []
    for index in similar_indices:
        similar_items.append(item_item_matrix.index[index])
    return similar_items

# Obtener recomendaciones para el ítem 'Garfield Kart'
recommended_items = get_similar_items('Borderlands 2', num_similar_items=5)

# Mostrar los ítems más similares al ítem 'Garfield Kart'
print(recommended_items)

["Zuma's Revenge!", 'Far Cry®', 'Feeding Frenzy 2 Deluxe', 'Final DOOM', 'FlatOut']


In [20]:
# Filtrar los datos relevantes (recommend=True y sentimientos válidos)
relevant_data = random_merged_recommend_model[(random_merged_recommend_model['recommend'] == True) & 
                          random_merged_recommend_model['sentiment_analysis'].isin([0, 1, 2])]

# Construir la matriz de ítem-ítem usando pivot_table
item_item_matrix = relevant_data.pivot_table(index='item_id', columns='app_name', values='sentiment_analysis', fill_value=0)

# Calcular la similitud de coseno entre ítems
item_similarity = cosine_similarity(item_item_matrix.T)

# Función para obtener ítems similares a un ítem dado por su nombre ('app_name')
def get_similar_items(item_name, num_similar_items=5):
    item_id = item_item_matrix.index.get_loc(item_name)
    similar_indices = item_similarity[item_id].argsort()[::-1][1:num_similar_items+1]
    similar_items = []
    for index in similar_indices:
        similar_items.append(item_item_matrix.index[index])
    return similar_items

# Obtener recomendaciones para el ítem 'Garfield Kart'
recommended_items = get_similar_items(49520, num_similar_items=5)

# Mostrar los ítems más similares al ítem 'Garfield Kart'
print(recommended_items)

# Filtrar el DataFrame por los item_id en recommended_items
filtered_names = random_merged_recommend_model[random_merged_recommend_model['item_id'].isin(recommended_items)]

# Seleccionar las columnas 'item_id' y 'app_name'
selected_names = filtered_names[['item_id', 'app_name']]

# Eliminar duplicados en 'item_id' y 'app_name'
unique_names = selected_names.drop_duplicates(subset='item_id').reset_index(drop=True)

# Mostrar los nombres únicos correspondientes a recommended_items
print('\n',unique_names)

[55230, 9450, 8930, 8980, 9010]

    item_id                                     app_name
0     8930                  Sid Meier's Civilization® V
1     8980                                  Borderlands
2     9010                 Return to Castle Wolfenstein
3     9450  Warhammer® 40,000: Dawn of War® - Soulstorm
4    55230                        Saints Row: The Third


In [21]:
# Paso 1: Filtrar datos relevantes
relevant_data = random_merged_recommend_model[(random_merged_recommend_model['recommend'] == True) & 
                                              random_merged_recommend_model['sentiment_analysis'].isin([0, 1, 2])]

# Paso 2: Construir matriz de usuario-ítem
user_item_matrix = relevant_data.pivot_table(index='user_id', columns='item_id', values='sentiment_analysis', fill_value=0)

# Paso 3: Calcular similitud entre usuarios
user_similarity = cosine_similarity(user_item_matrix)

# Paso 4: Recomendaciones para cada usuario
def get_user_recommendations(user_id, num_recommendations=5):
    # Obtener la fila correspondiente al usuario
    user_row = user_item_matrix.loc[user_id].values.reshape(1, -1)
    
    # Calcular similitud entre el usuario y otros usuarios
    similarity_scores = cosine_similarity(user_row, user_item_matrix.values)
    
    # Obtener los usuarios más similares (excluyendo al propio usuario)
    similar_users_indices = similarity_scores.argsort()[0][-num_recommendations-1:-1]
    
    # Obtener los ítems que los usuarios similares han recomendado
    recommended_items = user_item_matrix.iloc[similar_users_indices].sum().sort_values(ascending=False).index.tolist()
    
    return recommended_items

# Ejemplo: Obtener recomendaciones para el usuario con ID 1
user_id = 'thesnozberry'

recommendations = get_user_recommendations(user_id)
top_5_recommendations = recommendations[:5]

print(top_5_recommendations)

# Filtrar el DataFrame por los item_id en recommended_items
filtered_names = random_merged_recommend_model[random_merged_recommend_model['item_id'].isin(top_5_recommendations)]

# Seleccionar las columnas 'item_id' y 'app_name'
selected_names = filtered_names[['item_id', 'app_name']]

# Eliminar duplicados en 'item_id' y 'app_name'
unique_names = selected_names.drop_duplicates(subset='item_id').reset_index(drop=True)

# Mostrar los nombres únicos correspondientes a recommended_items
print('\n',unique_names)

[12830, 10, 28000, 26800, 25990]

    item_id                             app_name
0       10                       Counter-Strike
1    12830  Operation Flashpoint: Dragon Rising
2    25990                      Majesty Gold HD
3    26800                                Braid
4    28000             Kane & Lynch 2: Dog Days


In [22]:
def recomendacion_juego(user_id: str, num_recommendations=5):
    relevant_data = random_merged_recommend_model[(random_merged_recommend_model['recommend'] == True) &
                                                  random_merged_recommend_model['sentiment_analysis'].isin([0, 1, 2])]
    user_item_matrix = relevant_data.pivot_table(
        index='user_id', columns='item_id', values='sentiment_analysis', fill_value=0)
    user_similarity = cosine_similarity(user_item_matrix)
    user_row = user_item_matrix.loc[user_id].values.reshape(1, -1)
    similarity_scores = cosine_similarity(user_row, user_item_matrix.values)
    similar_users_indices = similarity_scores.argsort()[
        0][-num_recommendations-1:-1]
    recommended_items = user_item_matrix.iloc[similar_users_indices].sum(
    ).sort_values(ascending=False).index.tolist()

    return recommended_items


user_id = 'thesnozberry'

recommendations = recomendacion_juego(user_id)
top_5_recommendations = recommendations[:5]

In [30]:
filtered_data1 = random_merged_recommend_model[random_merged_recommend_model['item_id'].isin(['10', '20'])]
filtered_data1.head()

,item_id,app_name,recommend,sentiment_analysis,user_id


In [24]:
filtered_data = random_merged_recommend_model[random_merged_recommend_model['app_name'] != 'Counter-Strike']
filtered_data.head(1)  # Muestra los primeros 10 registros


,item_id,app_name,recommend,sentiment_analysis,user_id
120,20,Team Fortress Classic,True,1,hektoz


In [31]:
valores_unicos = random_merged_recommend_model['item_id'].unique()
print(valores_unicos)

[   10    20    30    40    50    60    70    80   130   220   240   280
   300   320   360   380   400   420   500   550   620   730  1200  1250
  1280  1510  1520  1900  1930  2100  2200  2270  2280  2290  2300  2310
  2320  2400  2420  2450  2500  2590  2600  2620  2630  2640  2700  2810
  2820  2870  2990  3170  3320  3390  3480  3483  3590  3620  3700  3720
  3730  3830  3900  3910  3920  4000  4500  4560  4570  4580  4700  4720
  4760  4920  6000  6020  6030  6060  6120  6220  6310  6370  6550  6830
  6850  6860  6870  6880  6900  6910  6980  7000  7510  7520  7600  7660
  7670  7730  7760  7770  7830  7860  7940  8000  8080  8140  8170  8190
  8310  8340  8400  8600  8800  8850  8870  8880  8930  8980  9010  9050
  9070  9160  9180  9200  9350  9420  9450  9460  9480  9500  9880  9900
  9930 10090 10130 10150 10180 10500 10680 11230 11240 11260 11280 11330
 11340 11360 11450 11610 12100 12110 12120 12140 12150 12160 12200 12220
 12340 12360 12810 12830 13210 13230 13250 13520 13